<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/BDSN_ShipCrew_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![CC-BY-SA](https://licensebuttons.net/l/by-sa/3.0/88x31.png)<br>
<hr>

![alt text](http://1.bp.blogspot.com/-nqAGzznZQNo/UwS8rxjfXeI/AAAAAAAABTA/nunmRLowpps/s1600/PraxisLogo.gif)<br>
[Data Science Program](http://praxis.ac.in/Programs/business-analytics/)

<hr>

[Prithwis Mukerjee](http://www.yantrajaal.com)<br>

# BDSN Ship Crews <br>
https://www.geeksforgeeks.org/pyspark-linear-regression-using-apache-mllib/

# Initialise

## Deprecated -- do not use

In [ ]:
#!apt-get update > /dev/null
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://apache.osuosl.org/spark/spark-2.2.2/spark-2.2.2-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
#wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
#!wget -q http://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
#
# if the current version of Spark is not used, there may be errors
# check here for current versions http://apache.osuosl.org/spark
#
#!tar xf spark-2.4.0-bin-hadoop2.7.tgz
#!tar xf spark-2.4.5-bin-hadoop2.7.tgz
#!tar xf spark-2.4.6-bin-hadoop2.7.tgz
#!tar xf spark-3.0.1-bin-hadoop3.2.tgz
#!tar xf spark-3.1.2-bin-hadoop3.2.tgz
#!pip install -q findspark
#!pip install -q pyspark

In [ ]:
#import os
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
#os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

## Install Spark, current version

In [1]:
#import findspark
#findspark.init()
!pip3 install -q pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Praxis_Shipping').master("local[*]").getOrCreate()
sc = spark.sparkContext
sc

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 55.4 MB/s 


<SparkContext master=local[*] appName=Praxis_Shipping>

# Load Data <br>
Data is available at https://drive.google.com/file/d/1fLGDQjsnA3RzTzzEVDpHxyy5ERzcV9Qw/view?usp=sharing

In [2]:
# Data from Google Drive
!gdown https://drive.google.com/uc?id=1fLGDQjsnA3RzTzzEVDpHxyy5ERzcV9Qw



Downloading...
From: https://drive.google.com/uc?id=1fLGDQjsnA3RzTzzEVDpHxyy5ERzcV9Qw
To: /content/ShippingCrewInfo.csv
100% 8.73k/8.73k [00:00<00:00, 12.9MB/s]


In [3]:
!ls -al

total 28
drwxr-xr-x 1 root root 4096 May 22 01:10 .
drwxr-xr-x 1 root root 4096 May 22 01:06 ..
drwxr-xr-x 4 root root 4096 May 17 13:38 .config
drwxr-xr-x 1 root root 4096 May 17 13:39 sample_data
-rw-r--r-- 1 root root 8734 May 22 01:10 ShippingCrewInfo.csv


In [5]:
crew_df=spark.read.csv('/content/ShippingCrewInfo.csv',inferSchema=True,header=True)
crew_df.show(10)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [6]:
#prints structure of dataframe along with datatype
crew_df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



# Pre Processing : Indexing, Vectorising

In [9]:
#columns identified as features are as below:
#['Cruise_line','Age','Tonnage','passengers','length','cabins','passenger_density']
#to work on the features, spark MLlib expects every value to be in numeric form
#feature 'Cruise_line is string datatype
#using StringIndexer, string type will be typecast to numeric datatype
#import library strinindexer for typecasting
  
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol='Cruise_line',outputCol='cruise_cat')
crew_df2 = indexer.fit(crew_df).transform(crew_df)
  
#above code will convert string to numeric feature and create a new dataframe
#new dataframe contains a new feature 'cruise_cat' and can be used further
#feature cruise_cat is now vectorized and can be used to fed to model
crew_df2.head(10)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0),
 Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, cruise_cat=1.0),
 Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0, cruise_cat=1.0),
 Row(Ship_name='Ecstasy', Cruise_line='Carnival', Age=22, Tonnage=70.367, passengers=20.52, length=8.55, cabins=10.2, pass

In [12]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
#creating vectors from features
#Apache MLlib takes input if vector form
assembler=VectorAssembler(inputCols=['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'cruise_cat'],outputCol='features')
crew_df3=assembler.transform(crew_df2)
crew_df3.head(5)
#crew_df3.select('features','crew').show(10)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0])),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0])),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0, features=DenseVector([26.0, 47.262, 14.86, 7.22, 7.43, 31.8, 1.0])),
 Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, cruise_cat=1.0, features=DenseVector([11.0, 110.0, 29.74, 9.53, 14.88, 36.99, 1.0])),
 Row(Ship_name='Destin

In [13]:
#final data consist of features and label which is crew.
data=crew_df3.select('features','crew')
#splitting data into train and test
train_data,test_data=data.randomSplit([0.7,0.3])
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              119|
|   mean| 8.02966386554623|
| stddev|3.599648272076911|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



# Regression
Read this https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.regression.LinearRegression.html

In [14]:
#import LinearRegression library
from pyspark.ml.regression import LinearRegression
#creating an object of class LinearRegression
#object takes features and label as input arguments
ship_lr=LinearRegression(featuresCol='features',labelCol='crew')
#pass train_data to train model
trained_ship_model=ship_lr.fit(train_data)
#evaluating model trained for Rsquared error
ship_results=trained_ship_model.evaluate(train_data)
  
print('Rsquared Error :',ship_results.r2)
#R2 value shows accuracy of model is 92%
#model accuracy is very good and can be use for predictive analysis

Rsquared Error : 0.9184773800514372


## Prediction

### With Unlabelled Data

In [15]:
#testing Model on unlabeled data
#create unlabeled data from test_data
#testing model on unlabeled data
unlabeled_data=test_data.select('features')
unlabeled_data.show(5)

+--------------------+
|            features|
+--------------------+
|[5.0,133.5,39.59,...|
|[6.0,110.23899999...|
|[6.0,112.0,38.0,9...|
|[8.0,77.499,19.5,...|
|[9.0,85.0,19.68,9...|
+--------------------+
only showing top 5 rows



In [16]:
predictions=trained_ship_model.transform(unlabeled_data)
predictions.show()
#below are the results of output from test data

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|[5.0,133.5,39.59,...| 13.218521195544367|
|[6.0,110.23899999...| 11.165823769337832|
|[6.0,112.0,38.0,9...| 11.352168081994238|
|[8.0,77.499,19.5,...|    8.6100480357337|
|[9.0,85.0,19.68,9...|  9.381625002454804|
|[10.0,110.0,29.74...| 12.101781193878638|
|[11.0,138.0,31.14...| 13.002529590222206|
|[12.0,2.329,0.94,...|0.36507082731101037|
|[12.0,58.6,15.66,...|  7.446980534755834|
|[12.0,77.104,20.0...|   8.80295538174027|
|[12.0,91.0,20.32,...|   9.29304074023379|
|[13.0,30.27699999...|  3.986201907271722|
|[13.0,63.0,14.4,7...|   6.71836528818074|
|[13.0,76.0,18.74,...|   8.79886835754881|
|[14.0,63.0,14.4,7...|  6.710471816399595|
|[14.0,83.0,17.5,9...|  9.351703519068403|
|[15.0,78.491,24.3...|  8.293107239570539|
|[15.0,83.338,17.5...|  9.346906596742734|
|[16.0,74.137,19.5...|  8.694822389025303|
|[16.0,78.491,24.3...|  8.285213767789395|
+----------

### With Labelled Data

In [20]:
# Generate predictions
predicted2 = trained_ship_model.transform(test_data)
predicted2.show()

+--------------------+-----+-------------------+
|            features| crew|         prediction|
+--------------------+-----+-------------------+
|[5.0,133.5,39.59,...|13.13| 13.218521195544367|
|[6.0,110.23899999...| 11.5| 11.165823769337832|
|[6.0,112.0,38.0,9...| 10.9| 11.352168081994238|
|[8.0,77.499,19.5,...|  9.0|    8.6100480357337|
|[9.0,85.0,19.68,9...| 8.69|  9.381625002454804|
|[10.0,110.0,29.74...| 11.6| 12.101781193878638|
|[11.0,138.0,31.14...|11.85| 13.002529590222206|
|[12.0,2.329,0.94,...|  0.6|0.36507082731101037|
|[12.0,58.6,15.66,...|  7.0|  7.446980534755834|
|[12.0,77.104,20.0...| 9.59|   8.80295538174027|
|[12.0,91.0,20.32,...| 9.99|   9.29304074023379|
|[13.0,30.27699999...|  4.0|  3.986201907271722|
|[13.0,63.0,14.4,7...| 5.31|   6.71836528818074|
|[13.0,76.0,18.74,...|  8.5|   8.79886835754881|
|[14.0,63.0,14.4,7...| 5.61|  6.710471816399595|
|[14.0,83.0,17.5,9...| 9.45|  9.351703519068403|
|[15.0,78.491,24.3...|  6.6|  8.293107239570539|
|[15.0,83.338,17.5..

In [21]:
trained_ship_model.setPredictionCol("PraxisPredictions!")

LinearRegressionModel: uid=LinearRegression_a0a5e36670ff, numFeatures=7

In [22]:
# Generate predictions
predicted3 = trained_ship_model.transform(test_data)
predicted3.show()

+--------------------+-----+-------------------+
|            features| crew| PraxisPredictions!|
+--------------------+-----+-------------------+
|[5.0,133.5,39.59,...|13.13| 13.218521195544367|
|[6.0,110.23899999...| 11.5| 11.165823769337832|
|[6.0,112.0,38.0,9...| 10.9| 11.352168081994238|
|[8.0,77.499,19.5,...|  9.0|    8.6100480357337|
|[9.0,85.0,19.68,9...| 8.69|  9.381625002454804|
|[10.0,110.0,29.74...| 11.6| 12.101781193878638|
|[11.0,138.0,31.14...|11.85| 13.002529590222206|
|[12.0,2.329,0.94,...|  0.6|0.36507082731101037|
|[12.0,58.6,15.66,...|  7.0|  7.446980534755834|
|[12.0,77.104,20.0...| 9.59|   8.80295538174027|
|[12.0,91.0,20.32,...| 9.99|   9.29304074023379|
|[13.0,30.27699999...|  4.0|  3.986201907271722|
|[13.0,63.0,14.4,7...| 5.31|   6.71836528818074|
|[13.0,76.0,18.74,...|  8.5|   8.79886835754881|
|[14.0,63.0,14.4,7...| 5.61|  6.710471816399595|
|[14.0,83.0,17.5,9...| 9.45|  9.351703519068403|
|[15.0,78.491,24.3...|  6.6|  8.293107239570539|
|[15.0,83.338,17.5..

#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)